---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [280]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [172]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [66]:
def get_list_of_university_towns():
    data = []
    state = []
    region = []
    with open("university_towns.txt") as file:
        for first in file:
            data.append(first[:-1])
        for line in data:
            if line[-6:] == '[edit]':
                line = line.strip("[edit]")
                state1 = line
            
            elif "(" in line:
                a = line.index("(")
                line = line.replace(line[a:], "")
                region.append(line.rstrip())
                state.append(state1)
                
        state_region = pd.DataFrame()
        state_region["State"] = state
        state_region["RegionName"] = region
        
    return state_region

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
...,...,...
508,Wisconsin,River Falls
509,Wisconsin,Stevens Point
510,Wisconsin,Waukesha
511,Wisconsin,Whitewater


In [270]:
def get_recession_start():
    start = pd.read_excel("gdplev.xls", skiprows=219)
    col_todrop = [0, 1, 2, 3, 5, 7]
    start.drop(start.columns[col_todrop], axis=1, inplace=True)
    start.rename(columns={"1999q4": "YearQ", 12323.3: "GDP"}, inplace=True)
    start.set_index("YearQ", inplace=True)
    
    star = []
    for i in range(0, len(start['GDP'])-1):
        if i > 3:
            if start['GDP'][i] < start['GDP'][i-1] and start['GDP'][i-1] < start['GDP'][i-2]:
                star.append(i-1)
                break
                
    ans = star[0]
    
    return start.index[ans]

get_recession_start()

'2008q3'

In [325]:
def get_recession_end():
    start = pd.read_excel("gdplev.xls", skiprows=219)
    col_todrop = [0, 1, 2, 3, 5, 7]
    start.drop(start.columns[col_todrop], axis=1, inplace=True)
    start.rename(columns={"1999q4": "YearQ", 12323.3: "GDP"}, inplace=True)
    start.set_index("YearQ", inplace=True)
    
    star = []
    for i in range(0, len(start['GDP'])-1):
        if i > 3:
            if start['GDP'][i] < start['GDP'][i-1] and start['GDP'][i-1] < start['GDP'][i-2] and start['GDP'][i+1] > start['GDP'][i] and start['GDP'][i+2] > start['GDP'][i+1]:
                star.append(i+2)
                break
                
    ans = star[0]
    
    return start.index[ans]

get_recession_end()

'2009q4'

In [100]:
def get_recession_bottom():
    start = pd.read_excel("gdplev.xls", skiprows=219)
    col_todrop = [0, 1, 2, 3, 5, 7]
    start.drop(start.columns[col_todrop], axis=1, inplace=True)
    start.rename(columns={"1999q4": "YearQ", 12323.3: "GDP"}, inplace=True)
    start.set_index("YearQ", inplace=True)
    
    star = []
    for i in range(0, len(start['GDP'])-1):
        if i > 3:
            if start['GDP'][i] < start['GDP'][i-1] and start['GDP'][i-1] < start['GDP'][i-2] and start['GDP'][i+1] > start['GDP'][i] and start['GDP'][i+2] > start['GDP'][i+1]:
                star.append(i)
                break
                
    ans = star[0]
    
    return start.index[ans]

get_recession_bottom()

'2009q2'

In [357]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    unitowns = get_list_of_university_towns()
    bottom = get_recession_bottom()
    start = get_recession_start()
    hdata = convert_housing_data_to_quarters()
    bstart = hdata.columns[hdata.columns.get_loc(start) - 1]
    
    hdata['ratio'] =  hdata[bstart] - hdata[bottom]
    hdata = hdata[[bottom, bstart, 'ratio']]
    hdata = hdata.reset_index()
    
    unitowns_hdata = pd.merge(hdata,unitowns,how='inner',on=['State','RegionName'])
    unitowns_hdata['uni'] = True
    hdata2 = pd.merge(hdata, unitowns_hdata, how='outer', on=['State','RegionName',bottom, bstart, 'ratio'])
    hdata2['uni'] = hdata2['uni'].fillna(False)
    
    ut = hdata2[hdata2['uni'] == True]
    nut = hdata2[hdata2['uni'] == False]
    
    t,p = ttest_ind(ut['ratio'].dropna(), nut['ratio'].dropna())
    
    different = True if p<0.01 else False
    better = "university town" if ut['ratio'].mean() < nut['ratio'].mean() else "non-university town"
    
    return(different, p, better)

run_ttest()

c:\python\python37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\python\python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(True, 0.008298688679786965, 'university town')

In [363]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    house_data = convert_housing_data_to_quarters()
    start = get_recession_start()
    bottom = get_recession_bottom()
    start1 = house_data.columns[house_data.columns.get_loc(start)-1]
    house_data['Difference'] = house_data[start1] - house_data[bottom]
    new = house_data['Difference']
    
    uni_data = get_list_of_university_towns()
    uni_data.set_index(['State', 'RegionName'], inplace=True)
    
    yes_uni = pd.merge(new, uni_data, left_index=True, right_index=True, how='inner')
    no_uni = pd.merge(new, uni_data, left_index=True, right_index=True, how='outer')
    yes_uni = yes_uni.dropna()
    no_uni = no_uni.dropna()
    
    s, p = ttest_ind(yes_uni['Difference'], no_uni['Difference'])
    
    if p < 0.01:
        different = True
    else:
        different = False
        
    if yes_uni['Difference'].mean() < no_uni['Difference'].mean():
        better = 'university town'
    else:
        better = 'non-university town'
    
    return (different, p, better)

run_ttest()

c:\python\python37\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
c:\python\python37\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


(True, 0.00952533183832617, 'university town')